In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import json

In [24]:
class BookingScraper():
    def __init__(self):
        self.driver = webdriver.Chrome()
        self.places = ['Międzyzdroje']
        # Do skrapowania ,'Międzyzdroje',
        #'Gdynia', 'Świnoujście','Kołobrzeg', 'Mielno','Łeba', 'Władysławowo'
        self.json_path = 'dataN4.json'
        self.CreateNewJsonFile()
        self.json_file = None
        self.ReadJson()
        
        
    def CreateNewJsonFile(self):
        with open(self.json_path, "w") as f:
            json.dump([], f)

    def ReadJson(self):
        with open(self.json_path, 'r') as f:
            self.json_file = json.load(f)

    def WriteToJson(self, dictionary_with_data):
        self.json_file.append(dictionary_with_data)
        with open(self.json_path, 'w') as f:
            json.dump(self.json_file, f)

    def Scrap_Booking(self):
        for place in self.places:
            self.Open_Website()
            self.Choose_Place(place)

    def Open_Website(self):
        self.driver.delete_all_cookies()
        self.driver.get('https://www.booking.com')
        time.sleep(1)
        cookie_button = self.driver.find_element(by=By.XPATH, value='//*[@id="onetrust-accept-btn-handler"]')
        cookie_button.click()
        self.driver.maximize_window()

    def HowManyPages(self):
        choose_tab_div = self.driver.find_elements(By.XPATH,'//*[@id="search_results_table"]/div[2]/div/div/div/div[7]/div[2]/nav/div/div[2]/ol/li') 
        return choose_tab_div[-1].text

    def Choose_Place(self, place):
        where_to_go = self.driver.find_element(by=By.XPATH, value='//*[@id="ss"]')
        where_to_go.click()
        where_to_go.send_keys(place)
        reservation_date_from = self.driver.find_element(by=By.XPATH, value='//*[@id="frm"]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/span')
        reservation_date_from.click()
        date_click = self.driver.find_element(by=By.XPATH, value='//*[@id="frm"]/div[1]/div[2]/div[2]/div/div/div[3]/div[2]/table/tbody/tr[1]/td[6]')
        date_click.click()
        date_click = self.driver.find_element(by=By.XPATH, value='//*[@id="frm"]/div[1]/div[2]/div[2]/div/div/div[3]/div[2]/table/tbody/tr[2]/td[6]')
        date_click.click()
        find_button = self.driver.find_element(by=By.XPATH, value='//*[@id="frm"]/div[1]/div[4]/div[2]/button')
        find_button.click()
        time.sleep(2)
        self.how_many_pages = int(self.HowManyPages())
        self.Scrap_One_Place(place)
        
    def Scrap_One_Place(self, place):
        for i in range(self.how_many_pages):
            self.Scrap_One_Tab(place)
            next_page_button = self.driver.find_element(By.XPATH, '//*[@id="search_results_table"]/div[2]/div/div/div/div[7]/div[2]/nav/div/div[3]/button')
            next_page_button.click()
            time.sleep(3)
     
    def Scrap_One_Tab(self, place):        
        hotels_whole_div = self.driver.find_elements(By.XPATH, '//*[@id="search_results_table"]/div/div/div/div/div[6]')
        hotels_whole_div = self.driver.find_elements(By.CSS_SELECTOR,'[data-testid="property-card"]')
        i = 0
        for hotels_div in hotels_whole_div:
                dictionary_with_data = {}
                hotel_button = hotels_div.find_element(By.CSS_SELECTOR,'[data-testid="title"]')
                hotel_price_div = hotels_div.find_element(By.CSS_SELECTOR,'[data-testid="price-and-discounted-price"]')
                
                hotel_price_div.find_element(By.TAG_NAME, 'span')
                hotel_price = hotel_price_div.text
                if (hotel_price.count("zł")) == 2:
                        hotel_price = hotel_price.split("zł")[1]
                
                hotel_name = hotel_button.text
                #print(hotel_name, place_distiny, hotel_price)
                
                hotel_button.click()
                time.sleep(4)
                
                self.driver.switch_to.window(self.driver.window_handles[1])
                dictionary_with_data.update({"hotel_name":hotel_name, "hotel_city":place, "hotel_price":hotel_price})
                self.Scrap_Data_About_Hotel(dictionary_with_data)
                self.driver.close()
                self.driver.switch_to.window(self.driver.window_handles[0])

    

    def Scrap_Data_About_Hotel(self, dictionary_with_data:dict):
        hotel_opinions = self.driver.find_elements(By.XPATH, '//*[@id="guest-featured_reviews__horizontal-block"]/div/div[5]/div/div/ul/li')
        for opinion in hotel_opinions:
            opinion_discribe = opinion.find_element(By.CLASS_NAME, 'c-score-bar__title').text
            opinion_value = opinion.find_element(By.CLASS_NAME, 'c-score-bar__score').text
            dictionary_with_data.update({opinion_discribe.strip():opinion_value})
            #print(opinion_discribe, opinion_value)

        hotels_facilities_list = self.driver.find_element(By.CLASS_NAME, 'hotel-facilities__list')
        hotels_facilities_list = hotels_facilities_list.find_elements(By.CLASS_NAME, 'bui-spacer--large')
        for hotel_facilitie in hotels_facilities_list:
            list_values_udo = []
            name_udo = hotel_facilitie.find_element(By.CLASS_NAME, 'bui-title__text').text
            elements_values_udo = hotel_facilitie.find_elements(By.CLASS_NAME, 'bui-list__description')
            for values_udo in elements_values_udo:
                list_values_udo.append(values_udo.text)
            #print(f'{name_udo}: {list_values_udo}')
            for item in list_values_udo:
                dictionary_with_data.update({f'{name_udo}:{item}':1})

        self.WriteToJson(dictionary_with_data)

In [25]:
scrap = BookingScraper()
scrap.Scrap_Booking()